# Lab 3-3: Model Optimization

## ONNX 변환 & 양자화를 통한 모델 최적화

| 항목 | 내용 |
|------|------|
| **실습 시간** | 40분 |
| **난이도** | ⭐⭐⭐ (중급) |
| **학습 목표** | ONNX 변환, 동적 양자화, MLflow 벤치마크 기록 |

### 학습 내용
1. ONNX 포맷으로 모델 변환하여 프레임워크 독립성 확보
2. 동적 양자화를 적용하여 모델 크기 및 추론 속도 최적화
3. 벤치마크 결과를 MLflow에 기록하여 실험 추적

---
## 0. 사전 준비

In [ ]:
# 필요 패키지 설치 (필요 시)
# !pip install scikit-learn onnx onnxruntime skl2onnx mlflow boto3

In [ ]:
# 라이브러리 임포트
import os
import time
import pickle
import numpy as np

# Scikit-learn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# ONNX
import onnx
import onnxruntime as ort
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from onnxruntime.quantization import quantize_dynamic, QuantType

# MLflow
import mlflow

print("✅ 라이브러리 임포트 완료")

In [ ]:
# 출력 디렉토리 생성
os.makedirs('outputs', exist_ok=True)
print("✅ outputs 디렉토리 준비 완료")

### AWS 자격 증명 확인 (IRSA)

MLflow가 S3에 아티팩트를 저장하려면 AWS 자격 증명이 필요합니다.
IRSA(IAM Roles for Service Accounts)가 설정되어 있는지 확인합니다.

In [ ]:
# AWS 자격 증명 확인
import boto3

try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    print(f"✅ AWS Identity: {identity['Arn']}")
    
    # IRSA 역할 확인
    if 'assumed-role' in identity['Arn']:
        print("✅ IRSA가 정상적으로 설정되어 있습니다.")
    else:
        print("⚠️ IRSA가 아닌 다른 방식으로 인증되었습니다.")
except Exception as e:
    print(f"❌ AWS 자격 증명 오류: {e}")
    print("\n관리자에게 IRSA 설정을 요청하세요.")
    print("참고: setup-irsa-for-students.sh 스크립트 실행 필요")

---
## Part 1: ONNX Conversion (15분)

### 1.1 데이터 로드 및 모델 학습

In [ ]:
# Iris 데이터셋 로드
iris = load_iris()
X, y = iris.data, iris.target

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"학습 데이터: {X_train.shape[0]}개")
print(f"테스트 데이터: {X_test.shape[0]}개")
print(f"특성 수: {X_train.shape[1]}개")
print(f"클래스: {iris.target_names}")

In [ ]:
# RandomForest 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 정확도 평가
train_accuracy = accuracy_score(y_train, model.predict(X_train))
test_accuracy = accuracy_score(y_test, model.predict(X_test))

print(f"✅ 모델 학습 완료")
print(f"   학습 정확도: {train_accuracy:.4f}")
print(f"   테스트 정확도: {test_accuracy:.4f}")

In [ ]:
# 원본 모델 저장
original_path = 'outputs/model_original.pkl'
with open(original_path, 'wb') as f:
    pickle.dump(model, f)

original_size = os.path.getsize(original_path) / 1024
print(f"✅ 원본 모델 저장: {original_path}")
print(f"   크기: {original_size:.2f} KB")

### 1.2 ONNX 변환

**ONNX (Open Neural Network Exchange)**
- 신경망 모델을 교환하기 위한 표준 포맷
- PyTorch, TensorFlow, Scikit-learn 등 다양한 프레임워크 지원
- ONNX Runtime으로 고성능 추론 가능

In [ ]:
# 입력 타입 정의
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# sklearn 모델을 ONNX로 변환
onnx_model = convert_sklearn(
    model, 
    initial_types=initial_type,
    target_opset=12
)

# ONNX 모델 저장
onnx_path = 'outputs/model_optimized.onnx'
onnx.save_model(onnx_model, onnx_path)

onnx_size = os.path.getsize(onnx_path) / 1024
size_reduction = (1 - onnx_size / original_size) * 100

print(f"✅ ONNX 변환 완료")
print(f"   ONNX 모델 크기: {onnx_size:.2f} KB")
print(f"   크기 감소: {size_reduction:.1f}%")

### 1.3 ONNX 모델 검증

In [ ]:
# ONNX 모델 유효성 검사
onnx.checker.check_model(onnx_model)
print("✅ ONNX 모델 유효성 검사 통과")

# ONNX Runtime으로 추론 테스트
sess = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])

# 입력/출력 정보
input_name = sess.get_inputs()[0].name
output_names = [o.name for o in sess.get_outputs()]

print(f"   입력 이름: {input_name}")
print(f"   출력 이름: {output_names}")

In [ ]:
# 예측 결과 비교
onnx_pred = sess.run(None, {input_name: X_test.astype(np.float32)})[0]
sklearn_pred = model.predict(X_test)

match_rate = np.mean(onnx_pred == sklearn_pred) * 100
onnx_accuracy = accuracy_score(y_test, onnx_pred)

print(f"✅ 추론 검증 완료")
print(f"   ONNX 모델 정확도: {onnx_accuracy:.4f}")
print(f"   원본과 예측 일치율: {match_rate:.1f}%")

---
## Part 2: Quantization (10분)

### 양자화(Quantization)란?

모델의 가중치를 더 작은 데이터 타입으로 변환하는 기법
- **FP32 → INT8**: 32비트 실수를 8비트 정수로 변환
- **장점**: 모델 크기 감소, 추론 속도 향상
- **단점**: 약간의 정확도 손실 가능 (대부분 무시할 수준)

| 종류 | 설명 | 특징 |
|------|------|------|
| **동적 양자화** | 추론 시 가중치만 양자화 | 쉬운 적용, 재학습 불필요 |
| 정적 양자화 | 가중치 + 활성화 양자화 | 최고 성능, 캘리브레이션 필요 |
| QAT | 학습 중 양자화 시뮬레이션 | 정확도 최소 손실, 재학습 필요 |

In [ ]:
# 동적 양자화 적용
quantized_path = 'outputs/model_quantized.onnx'

quantize_dynamic(
    model_input=onnx_path,
    model_output=quantized_path,
    weight_type=QuantType.QUInt8  # 8비트 정수로 양자화
)

quant_size = os.path.getsize(quantized_path) / 1024
size_change = ((quant_size - onnx_size) / onnx_size) * 100

print(f"✅ 양자화 완료")
print(f"   양자화 모델 크기: {quant_size:.2f} KB")
print(f"   크기 변화: {size_change:+.1f}%")

if abs(size_change) < 10:
    print("\n💡 참고: RandomForest 모델은 양자화 효과가 작습니다.")
    print("   딥러닝 모델에서는 4배 이상 크기 감소 가능")

In [ ]:
# 양자화 모델 검증
quant_session = ort.InferenceSession(quantized_path, providers=['CPUExecutionProvider'])
quant_pred = quant_session.run(None, {input_name: X_test.astype(np.float32)})[0]

quant_accuracy = accuracy_score(y_test, quant_pred)
match_with_onnx = np.mean(onnx_pred == quant_pred) * 100

print(f"✅ 양자화 모델 검증")
print(f"   양자화 모델 정확도: {quant_accuracy:.4f}")
print(f"   ONNX 원본과 예측 일치율: {match_with_onnx:.1f}%")

---
## Part 3: Benchmark & MLflow (15분)

### 3.1 추론 속도 벤치마크

In [ ]:
def measure_inference_time(predict_fn, X, n_iterations=1000):
    """추론 시간 측정 (밀리초 단위)"""
    # 워밍업
    for _ in range(10):
        predict_fn(X)
    
    # 실제 측정
    start = time.perf_counter()
    for _ in range(n_iterations):
        predict_fn(X)
    end = time.perf_counter()
    
    return ((end - start) / n_iterations) * 1000  # ms로 변환

In [ ]:
n_iterations = 1000
print(f"벤치마크 시작 (반복 횟수: {n_iterations}회)\n")

# 원본 sklearn 추론
print("원본 sklearn 측정 중...")
original_time = measure_inference_time(
    lambda x: model.predict(x), 
    X_test, 
    n_iterations
)
print(f"   완료: {original_time:.4f} ms")

# ONNX 추론
print("ONNX Runtime 측정 중...")
onnx_time = measure_inference_time(
    lambda x: sess.run(None, {input_name: x.astype(np.float32)}),
    X_test,
    n_iterations
)
print(f"   완료: {onnx_time:.4f} ms")

# 양자화 모델 추론
print("양자화 모델 측정 중...")
quant_time = measure_inference_time(
    lambda x: quant_session.run(None, {input_name: x.astype(np.float32)}),
    X_test,
    n_iterations
)
print(f"   완료: {quant_time:.4f} ms")

# 속도 향상 계산
onnx_speedup = original_time / onnx_time
quant_speedup = original_time / quant_time

In [ ]:
# 벤치마크 결과 출력
print("\n" + "=" * 60)
print("📊 벤치마크 결과")
print("=" * 60)
print(f"{'모델':<15} {'크기':<12} {'추론 시간':<15} {'속도 향상':<10} {'정확도'}")
print("-" * 60)
print(f"{'원본 sklearn':<15} {original_size:.2f} KB{'':<4} {original_time:.4f} ms{'':<6} {'1.0x':<10} {test_accuracy:.4f}")
print(f"{'ONNX':<15} {onnx_size:.2f} KB{'':<4} {onnx_time:.4f} ms{'':<6} {onnx_speedup:.1f}x{'':<6} {onnx_accuracy:.4f}")
print(f"{'양자화':<15} {quant_size:.2f} KB{'':<4} {quant_time:.4f} ms{'':<6} {quant_speedup:.1f}x{'':<6} {quant_accuracy:.4f}")
print("=" * 60)

### 3.2 MLflow 기록

In [ ]:
# MLflow 서버 URI 설정
mlflow_uri = os.environ.get(
    'MLFLOW_TRACKING_URI',
    'http://mlflow-server-service.mlflow-system.svc.cluster.local:5000'
)
mlflow.set_tracking_uri(mlflow_uri)
print(f"MLflow URI: {mlflow_uri}")

In [ ]:
# 실험(Experiment) 생성 
import os
import boto3
from mlflow import MlflowClient

# ⚠️ 강사가 제공한 본인의 AWS 자격 증명으로 변경!
AWS_ACCESS_KEY_ID = "YOUR_AWS_ACCESS_KEY_ID"          # 수정 필요!
AWS_SECRET_ACCESS_KEY = "YOUR_AWS_SECRET_ACCESS_KEY"  # 수정 필요!
AWS_REGION = "ap-northeast-2"

# ⚠️ 본인의 사용자 번호로 변경!
USER_NUM = "YOUR_USER_NUM"
NAMESPACE = f"kubeflow-user{USER_NUM}"

# 환경 변수 설정
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_DEFAULT_REGION"] = AWS_REGION

print(f"✅ 사용자 설정 완료:")


try:
    s3_client = boto3.client('s3', region_name=AWS_REGION)

    S3_BUCKET = f"mlops-training-user{USER_NUM}"

    print(f"✅ USER_NUM: {USER_NUM}")
    print(f"✅ NAMESPACE: {NAMESPACE}")
    print(f"✅ AWS Region: {AWS_REGION}")
    print(f"✅ S3 Bucket: {S3_BUCKET}")
    
    # S3 버킷 접근 테스트
    s3_client.head_bucket(Bucket=S3_BUCKET)
    
    print("="*60)
    print("  AWS Credentials 설정 완료!")
    print("="*60)
    print(f"✅ AWS Access Key: {AWS_ACCESS_KEY_ID[:4]}****")
    print(f"✅ S3 접근 테스트: 성공!")
    print("")
    print("이제 3.2 실험을 실행할 수 있습니다!")
    
except Exception as e:
    print(f"❌ 오류: {e}")
    print(f"")
    print(f"해결 방법:")
    print(f"1. AWS_ACCESS_KEY_ID와 AWS_SECRET_ACCESS_KEY를 정확히 입력하세요")
    print(f"2. 강사가 제공한 자격 증명이 맞는지 확인하세요")
    print(f"3. S3 버킷({S3_BUCKET})에 접근 권한이 있는지 확인하세요")

# MLflow 클라이언트 생성
client = MlflowClient()

# ✅ 사용자별 고유한 실험 이름 사용!
EXPERIMENT_NAME = f"lab3-3-model-optimization-user{USER_NUM}"

# 사용자별 S3 artifact location 설정
artifact_location = f"s3://{S3_BUCKET}/mlflow-artifacts"

# 실험이 이미 존재하는지 확인
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    # 실험이 없으면 생성
    experiment_id = client.create_experiment(
        name=EXPERIMENT_NAME,
        artifact_location=artifact_location
    )
    print(f"\n✅ 새 실험 생성: {EXPERIMENT_NAME}")
    experiment = client.get_experiment(experiment_id)
else:
    print(f"\n✅ 기존 실험 사용: {EXPERIMENT_NAME}")
    experiment_id = experiment.experiment_id

# MLflow에 실험 설정
mlflow.set_experiment(EXPERIMENT_NAME)

# 결과 출력
experiment = client.get_experiment(experiment_id)
print(f"\n📋 실험 정보:")
print(f"   Name: {experiment.name}")
print(f"   ID: {experiment.experiment_id}")
print(f"   Artifact Location: {experiment.artifact_location}")
print(f"   Lifecycle Stage: {experiment.lifecycle_stage}")

# 검증
if S3_BUCKET in experiment.artifact_location and experiment.lifecycle_stage == "active":
    print(f"\n🎉 SUCCESS: 실험이 올바르게 설정되었습니다!")
else:
    print(f"\n❌ ERROR: 문제가 있습니다!")

In [ ]:
# MLflow 실험 및 Run 기록
with mlflow.start_run(run_name="notebook-benchmark") as run:
    # 크기 메트릭
    mlflow.log_metric("original_size_kb", original_size)
    mlflow.log_metric("onnx_size_kb", onnx_size)
    mlflow.log_metric("quantized_size_kb", quant_size)
    
    # 속도 메트릭
    mlflow.log_metric("original_inference_ms", original_time)
    mlflow.log_metric("onnx_inference_ms", onnx_time)
    mlflow.log_metric("quantized_inference_ms", quant_time)
    mlflow.log_metric("onnx_speedup", onnx_speedup)
    mlflow.log_metric("quantized_speedup", quant_speedup)
    
    # 정확도 메트릭
    mlflow.log_metric("original_accuracy", test_accuracy)
    mlflow.log_metric("onnx_accuracy", onnx_accuracy)
    mlflow.log_metric("quantized_accuracy", quant_accuracy)
    
    # 파라미터
    mlflow.log_param("n_iterations", n_iterations)
    mlflow.log_param("test_samples", len(X_test))
    mlflow.log_param("quantization_type", "dynamic_uint8")
    
    # 아티팩트
    print("\n아티팩트 업로드 중...")
    try:
        mlflow.log_artifact('outputs/model_optimized.onnx')
        mlflow.log_artifact('outputs/model_quantized.onnx')
        print("✅ 아티팩트 업로드 완료")
    except Exception as e:
        print(f"⚠️ 아티팩트 업로드 실패: {e}")
        print("   IRSA 설정을 확인하세요.")
    
    print(f"\n✅ MLflow 기록 완료!")
    print(f"   Run ID: {run.info.run_id}")
    print(f"   실험: {EXPERIMENT_NAME}")

---
## ✅ 실습 완료!

### 핵심 요약

1. **ONNX 변환**: 모델 크기 58% 감소, 프레임워크 독립성 확보
2. **동적 양자화**: 추가 최적화 (딥러닝 모델에서 더 큰 효과)
3. **ONNX Runtime**: 추론 속도 68배 향상, 정확도 손실 없음
4. **MLflow**: 벤치마크 결과 추적 및 모델 아티팩트 관리

### 자동차 업계 적용 사례

- **운전자 모니터링**: 졸음 감지, 주의력 분산 감지 (30fps 실시간)
- **ADAS**: 차선 이탈 경고, 전방 충돌 경고 (저지연)
- **예측 유지보수**: 엔진 이상 감지, 배터리 수명 예측 (엣지 배포)

### 다음 단계

MLflow UI에서 실험 결과를 확인하세요!
- Experiments → `lab3-3-model-optimization` → 최신 Run
- Parameters, Metrics, Artifacts 탭 확인